In [1]:
%load_ext autoreload
%autoreload

import os
import sys
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

In [2]:
import getch.models as m

In [3]:
#m.MaskBase.objects.all().delete()

In [3]:
m.MaskBase.objects.all()

<QuerySet []>

# MaskBase

In [4]:
dir_masks = Path(parentdir) / 'getch/static/materials/masks'

In [5]:
for d in dir_masks.glob('*'):
    if d.name != 'archives':
        _type = d.name.upper()
        
        for f in d.rglob('*.*'):
            cat = f.parts[-2]
            
            if cat != 'raw':
                maskbase = m.MaskBase.objects.create(type=_type, category=cat)
                maskbase.pix.save(f.name, ContentFile(f.read_bytes()), save=True)

# MaskSerializer

In [6]:
class MaskSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.Mask
        fields = ['id', 'maskbase', 'top', 'left', 'width', 'height']
        read_only_fields = ['id']
        
MaskSerializer()

MaskSerializer():
    id = IntegerField(read_only=True)
    maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
    top = FloatField(required=False)
    left = FloatField(required=False)
    width = FloatField(required=False)
    height = FloatField(required=False)

## Serializing

In [7]:
mask = m.Mask.objects.all()[0]
data = MaskSerializer(mask).data; data

{'id': 2, 'maskbase': 14, 'top': 0.0, 'left': 0.0, 'width': 100.0, 'height': 20.0}

## Deserializing
### 1. Create

In [24]:
serializer = MaskSerializer(data={'maskbase':15, 'top':10, 'left':0, 'width':50, 'height':50})
serializer.is_valid()
mask = serializer.save(); mask

<Mask: EYE | animal | 2020-08-07_110328.890580SIDEB_EYEMASK_A_ANIMAL_02_cat.jpg | T=10.0, L=0.0, W=50.0, H=50.0>

### 2. Update

In [25]:
serializer = MaskSerializer(mask, data={'maskbase':16, 'top':0, 'left':0, 'width':50, 'height':50})
serializer.is_valid()
mask = serializer.save(); mask

<Mask: EYE | animal | 2020-08-07_110330.219042SIDEB_EYEMASK_A_ANIMAL_02_dog.jpg | T=0.0, L=0.0, W=50.0, H=50.0>

# ProfileSerializer

In [8]:
class ProfileSerializer(serializers.ModelSerializer):
    eye_mask = MaskSerializer(required=False)
    mouth_mask = MaskSerializer(required=False)

    class Meta:
        model = m.Profile
        fields = ['id', 'type', 'pix', 'image', 'text', 'eye_mask', 'mouth_mask']
        read_only_fields = ['id']
        
ProfileSerializer()

ProfileSerializer():
    id = IntegerField(read_only=True)
    type = ChoiceField(choices=(('IMAGE', 'image'), ('TEXT', 'text')), required=False)
    pix = ImageField(max_length=500)
    image = ImageField(allow_null=True, max_length=500, required=False)
    text = CharField(allow_blank=True, allow_null=True, max_length=10, required=False)
    eye_mask = MaskSerializer(required=False):
        id = IntegerField(read_only=True)
        maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
        top = FloatField(required=False)
        left = FloatField(required=False)
        width = FloatField(required=False)
        height = FloatField(required=False)
    mouth_mask = MaskSerializer(required=False):
        id = IntegerField(read_only=True)
        maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
        top = FloatField(required=False)
        left = FloatField(required=False)
        width = FloatField(required=False)
        height = FloatField(requ

## Serializing

In [14]:
profile = m.Profile.objects.last()
data = ProfileSerializer(profile).data; data

{'id': 8, 'type': 'IMAGE', 'pix': 'https://storage.googleapis.com/sideb-proejct.appspot.com/user/gem763%40gmail.com/2020-08-04_134305.193770__killyou.gif?Expires=1597127094&GoogleAccessId=sideb-serviceaccount%40sideb-proejct.iam.gserviceaccount.com&Signature=R4AlbT3eCphmIbVR1Vc4EBaD779rlAeqCHACK2CmHLSTiK6wWYM8tth8n4kmRhnt3ndRv%2Bn78N%2B304ezDG%2BiimpiYR3SOtSzXT%2FLttzRomu6d94coa1WRjgxgNLu%2BJ%2F7EDWnX8RagxLaAxtPCY%2BKf6AN59VN6%2BEY0YtARjcvNoSR2upvF0genDVNKJEZzfosC3AVZZeq%2BoB0xwWiR9tCVlvoshYgZIiZ%2B5QO%2B4TaKPJs55DkYZp%2BGHPWPDiW4Bi1XgYCtqOFMSy%2FKyeGa8sN25M72JwewucrbD7YarkXX3EZNxdhfnqSzqW3yv6D0BU6pTclPF1OhyYyebQO9iTZvw%3D%3D', 'image': None, 'text': None, 'eye_mask': OrderedDict([('id', 2), ('maskbase', 14), ('top', 0.0), ('left', 0.0), ('width', 100.0), ('height', 20.0)]), 'mouth_mask': None}

## Deserializing
### 1. create

In [15]:
pix = m.PostVoteOX.objects.all()[0].pix; pix

<ImageFieldFile: post/2020/7/22/sejin76@nate.com/2020-07-22_195937.085170__AB_anorak_B.jpg>

In [17]:
serializer = ProfileSerializer(data={'type':'IMAGE', 'text':'TESTTEST', 'pix':None, 'image':None, 'eye_mask':None, 'mouth_mask':None})
serializer.is_valid()
# profile = serializer.save(); profile

False

In [19]:
serializer.validated_data

{}

In [9]:
_pix = m.Profile.objects.first().pix; _pix.name

'user/test/2020-08-10_181145.027962__einstein.jpg'

In [10]:
profile = m.Profile.objects.create(pix=_pix); profile

<Profile: 8>